# Speed of computation

Python, being an interpreted language, tends to be slower than compiled languages like C or Fortran.  Some other languages like Java and Julia tend to use Just-in-Time compilation which can give speedups, but Python also has the problem of being dynamically typed, which eliminates the possibility of many optimizations.

The `timeit` library provides functions to estimate the time taken to run a piece of code.  It can automatically run the code multiple times to get better average results, and can be used to identify bottlenecks in your program.  However, it should be used with care as it is not a detailed function-call-level profiler.

It can either be `import`ed as a module where you can then explicitly called `timeit.timeit(func)` to estimate time for a function, or you can use the *magic syntax* in Python notebooks as shown below.  

import numpy as np
x = np.random.rand(10000,1)

In [58]:
def sumarr(x):
    sum = 0
    for i in range(len(x)): 
    # for i in x:
        sum += x[i]
    return sum
print(sumarr(x))
%timeit sumarr(x)

NameError: name 'x' is not defined

In [ ]:
import numpy as np
def npsumarr(x):
    return np.sum(x)
print(npsumarr(x))
%timeit npsumarr(x)

# Solving equations by Gaussian elimination

Once you have constructed two matrices A and B to represent the system of linear equations 
$$ Ax = b $$
you can then proceed to solve the equations using the process known as Gaussian elimination.

It is assumed you already know how the process works, but to refresh your memory, you could use the reference material at [LibreTexts](https://math.libretexts.org/Bookshelves/Algebra/Book%3A_Algebra_and_Trigonometry_(OpenStax)/11%3A_Systems_of_Equations_and_Inequalities/11.06%3A_Solving_Systems_with_Gaussian_Elimination).

Basically it involves making the A matrix *triangular* and ultimately into the shape of an identity matrix.

In [ ]:
# Input matrices - the set of equations - 2 variables x1 and x2
A = [ [2,3], [1,-1] ]
B = [6,1/2]
print(A)
print(B)

In [ ]:
# Normalize row 1
norm = A[0][0]
for i in range(len(A[0])): A[0][i] /= norm
B[0] = B[0]/norm
print(A)
print(B)

In [ ]:
# Eliminate row 2 - A[1] - need to check and ensure div-by-zero etc doesnt happen
norm = A[1][0] / A[0][0]
for i in range(len(A[1])): A[1][i] = A[1][i] - A[0][i] * norm
B[1] = B[1] - B[0] * norm
print(A)
print(B)

In [ ]:
# Normalize row 2 - B[1] will now contain the solution for x2
norm = A[1][1]
for i in range(len(A[1])): A[1][i] = A[1][i] / norm
B[1] = B[1] / norm
print(A)
print(B)

In [ ]:
# Sub back and solve for B[0] <-> x1
# This can be seen as eliminating A[0][1]
norm = A[0][1] / A[0][0]
# note that len(A) will give number of rows
for i in range(len(A)): 
    A[i][1] = A[i][1] - A[i][0] * norm
    B[i] = B[i] - A[i][0] * norm
print(A)
print(B)

## Problems with Gaussian elimination

There are several obvious problems with the method outlined here.  These include:

- Performance - Python lists are not the most efficient way to store matrices
- Zeros: the simple example does not consider a scenario where one of the values on the diagonal may be 0.  Then some shuffling of rows is required.
- Numerical stability: there are several *normalization* steps involved, where it is quite possible for the values to blow up out of control if not managed properly.  Usually some kind of pivoting techniques are used to get around these issues.

In [ ]:
import numpy as np
A1 = np.array( [ [2,3], [1,-1] ] )
B1 = np.array( [6, 1/2] )
np.linalg.solve(A1, B1)

# Library functions

*Numeric Python* or `numpy` is a library that allows Python code to directly call highly efficient implementations of several linear algebra routines (that have been written and optimized using C or Fortran and generally offer very high performance).

Although you can use the same `list` based approach to create matrices, it is better to declare them as the `array` data type:

- the numeric `type` (float, int etc.) can be specified for arrays
- memory allocation is done more efficiently by assuming they are not meant to be arbitrarily extended or changed

# SPICE basics

Our goal is to implement a SPICE simulator.  In order to do this, we first need to read in the circuit description from a text file.  To start with, we will only consider the basic elements of SPICE: Voltage sources, Current sources, and Resistors.  A typical SPICE netlist would look like this:

```spice
.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
```

This is basically a *netlist* with 3 *nodes* - one of which is Ground (GND) which is assumed to be have a voltage of 0V.  We can write down Kirchhoff's current law (KCL) equations at each node, to account for current balance.  In addition, we will have some equations that specify the voltages between nodes having a direct voltage source, since there is no resistance there to provide an equation.

For the above example, the equations will be

$$
\begin{aligned}
\frac{V1-0}{R1} & + & \frac{V1-V2}{R2} & = & 0 \\
\frac{V2-V1}{R2} & + & I1 & = & 0 \\
V2 & - & 0 & = & 2
\end{aligned}
$$

which can be written in Matrix form as:

$$
\begin{bmatrix}
\frac{1}{R1}+\frac{1}{R2} & \frac{-1}{R2} & 0 \\
\frac{-1}{R2}   & \frac{1}{R2}  & 1 \\
0  & 1  & 0
\end{bmatrix}
\begin{bmatrix}
V1 \\
V2 \\
I1
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0 \\
2
\end{bmatrix}
$$

At this point, you have reduced the solving of the SPICE equations to a known problem (linear equation solving) that you already know how to do.

element <+ve node> <-ve node> <value>


## AC sources

The assumption above is that the system consists only of Voltage or Current sources and resistors.  What about capacitors, inductors, and AC sources?  These can be handled in exactly the same way as long as the circuit is operating at a single frequency.  We then replace the elements with their corresponding *impedance* values, which are frequency dependent complex numbers, but since there is only a single frequency they will still be constants.

## Problem scenarios

- Voltage source loops
- Current sources at a node
- Circuit defined with both DC and AC sources
- Syntax errors

# String and File manipulation

Given a SPICE netlist like the one above, you need to *read* it and construct an internal matrix as described above.  For string manipulation, there are a few helpful utility functions that we can see here.

In [ ]:
circ = """.circuit
R1 GND 1 1  
R2 1 2 1    
V1 GND 2 dc 2
.end
""".splitlines()
for l in circ:
    if l[0] == 'R':
        print("Found a resistor")
    elif l[0] == 'V':
        print("Found a voltage source with value: ", float(l.split()[4]))

## Files

You can read from a file using the `readlines()` method of file objects.  One thing to keep in mind is how you open and close file objects.  In particular, it is strongly recommended to use the pattern `with open("filename") as f:` to ensure that the file is closed once you are done with reading it.  

# Assignment

The following are the problems you need to solve for this assignment.  You need to submit your code (either as standalone Python script or a Python notebook), a PDF document explaining your solution (either generated from the notebook or a separate LaTeX document), and any supporting files you may have (such as circuit netlists you used for testing your code).

- Write a function to find the factorial of N (N being an input) and find the time taken to compute it.  This will obviously depend on where you run the code and which approach you use to implement the factorial.  Explain your observations briefly.
- Write a linear equation solver that will take in matrices $A$ and $b$ as inputs, and return the vector $x$ that solves the equation $Ax=b$.  Your function should catch errors in the inputs and return suitable error messages for different possible problems.
  - Time your solver to solve a random $10\times 10$ system of equations.  Compare the time taken against the `numpy.linalg.solve` function for the same inputs.
- Given a circuit netlist in the form described above, read it in from a file, construct the appropriate matrices, and use the solver you have written above to obtain the voltages and currents in the circuit.  If you find AC circuits hard to handle, first do this for pure DC circuits, but you should be able to handle both voltage and current sources.

## Bonus assignments

- (Small bonus): after reading in the netlist, allow some or all sources and impedances to be controlled interactively - either using widgets or other mechanisms.  On each change you should recompute the currents and voltages and display them.
- (Large bonus): make a solver that can do real-time transient simulations of a SPICE netlist and update the currents and voltages dynamically.  They should also be plotted as a function of time and react to changes.  This is something along the lines of https://www.falstad.com/circuit/.  Ideally you should be able to do a real-time demo of some experiments you might conduct as part of a basic electronics lab, and simulate the behaviour of an oscilloscope and signal generator.

In [ ]:
#Code to Generate N factorial

def factorial(N):
    if N==1:
        return N
    else:
        return N*factorial(N-1)
    
print(factorial(15))
%timeit factorial(15)

In [ ]:
def fact(N):
    num = 1
    for i in range(N):
        num = num*(i+1)
    return num

print(fact(15))
%timeit fact(15)

* For finding factorial of number I have used reccursion method and a for loop method,in reccursion method the fuction is called again and again until N equals 1, this method usese the property N! = N*(N-1)! and solves the problem 
* The second method I have used is just used a simple for loop to calculate N! it uses the property N! = 1*2*3...(N-1)*N 
* The time of reccusive function is 984ns greater then for loop implementation of 665ns as reccursive fuction is calling the fuction again and again which takes time greater then  the for loop implementation; for loop implementation calls the fuction ones and multiplies the values till N and returns the answer and thus is relatively faster then reccursive implementation

In [75]:
#Gauss Elemination method.
import numpy as np
def row_swap(A,r1,r2):
    A[r1],A[r2] = A[r2],A[r1]
      
def agumented(A,b):
    n = len(A)
    for i in range(len(A)):
        A[i].append(b[i])
        
def matminor(A,i,j): 
    B = []
    for row in A[:i]+A[i+1:]:
        B.append(row[:j] + row[j+1:])
    return B

def det(A):
    determinant = 0
    #------------------------------------------------------------------
    if len(A) == 1:
        #-------------
        return A[0][0]
    elif len(A) == 2:
        #-------------------------------------
        return A[0][0]*A[1][1]-A[0][1]*A[1][0]
    else:
        for k in range(len(A)):
            determinant += (pow(-1,k))*A[0][k]*det(matminor(A,0,k))
        #-----------------
        return determinant
    
def normalization(A,c):
    m = len(A[0])
    n = len(A)
    #n = len(A)
    try:
        for i in range(m):
            if A[c][c] != 1:
                norm = A[c][c] 
                print(norm)
                for j in range(m):
                    A[c][j] = A[c][j]/norm
    except ZeroDivisionError:
        for i in range(n):
            for j in range(i+1,n):
                if A[j][i] != 0:
                    #swap A[j] with A[i]
                    row_swap(A,j,i)
        normalization(A,c)
                
                
def elemination(A,r,c):
    #normalization(A,c)
    m = len(A[0])
    for i in range(m):
        if A[r][c] != 0:
            ele = A[r][c]
            for j in range(m):
                A[r][j] = A[r][j] - ((ele)*A[c][j])

def gauss(A,b):
    agumented(A,b)
    m = len(A)
    for i in range(m):
        normalization(A,i)
        for j in range(m):
            if i != j:
                elemination(A,j,i)
    return A

def solutions(A,b):
    n = len(b)
    #print(A)
    d = det(A)
    if d == 0 and n == b.count(0):
        return ('Infinite Solutions')
    elif d == 0 and n != b.count(0):
        return ('No Solutions')
    else:
        sol = []
        B = gauss(A,b)
        print(np.array(B))
        for i in range(len(B)):
            sol.append(B[i][-1])

        return sol



#A = [[0,0,0],[2,0,-3],[3,6,-5]]
#b = [9,1,0]
#A = [[1,1,2],[2,4,-3],[3,6,-5]]
#print(np.array(gauss(A,b))) 
#print(solutions(A,b))
#A = [[0,1,2,5],[3,4,5,5],[6,7,8,5]]
# A = [[0,0,0],[1,0,3],[2,1,2]]
# b = [0,0,0]
# print(np.array(A))
# print(np.array(b))
# print(solutions(A,b))
# row_swap(A,1,2)
# print(np.array(normalization(A,0)))
# print(np.array(A))

A = np.random.rand(10,10).tolist()
b = np.random.rand(10,1).tolist()
#print(np.array(b))
#print(np.linalg.solve(np.array(A), np.array(b)).tolist())
#%timeit np.linalg.solve(np.array(A), np.array(b)).tolist()
print(solutions(A,b))
%timeit solutions(A,b)
#print(solutions(A,b))
#print(np.array(b))

0.9962570144967405


TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [ ]:
import numpy as np
def matminor(A,r,c): 
    B = []
    for row in A[:r] + A[r+1:]: 
        B.append(row[:c] + row[c+1:])
    return B


def det(A):
    determinant = 0
    #------------------------------------------------------------------
    if len(A) == 1:
        #-------------
        return A[0][0]
    elif len(A) == 2:
        #-------------------------------------
        return A[0][0]*A[1][1]-A[0][1]*A[1][0]
    else:
        for k in range(len(A)):
            determinant += (pow(-1,k))*A[0][k]*det(matminor(A,0,k))
        #-----------------
        return determinant
            
def tran(A):
    n = len(A)
    B = [[0] * n for i in range(n)]
    #--------------------------------
    for i in range(n):
        for j in range(n):
            B[j][i] = A[i][j]
    #--------------------------------
    return B

def inv(A):
    n = len(A)
    deter = det(A)
    #------------------------------------------------
    if len(A) == 2:
        return [[A[1][1]/deter, -1*A[0][1]/deter],
                [-1*A[1][0]/deter, A[0][0]/deter]]
    else:
        co = []
        #---------------------------------------------
        for i in range(n):
            corow = []
            for j in range(n):
                minor = matminor(A,i,j)
                corow.append(pow(-1,i+j)*det(minor))
            co.append(corow)
        co = tran(co)
        #----------------------------------------------
        for i in range(n):
            for j in range(n):
                co[i][j] = co[i][j]/deter
        #----------------------------------------------       
        return co

def matmult(A,B):
    n = len(A)
    m = len(B[0])
    result = [[0] * m for i in range(n)]
    #-------------------------------------------
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    #--------------------------------------------           
    return result
    
def  matsolver(A,b):
    A_inverse = inv(A)
    result = matmult(A_inverse,b)
    return result
        
    
            

#A = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
#A = [[1,2,3],[4,5,5],[7,8,9]]
#B = [[1,2,3],[4,5,6],[7,8,9]]
#print(det(A))
#print(np.array(A))
#print(np.array(matminor(A,3,2)))
#A = [[1,2],[3,4]]
#print(matminor(A,0,0))
#print(np.array(inv(A)))
#print(matmult(A,B))
#A = [ [2,3], [1,-1] ]
#B = [[6],[1/2]]
# print(matsolver(A,B))
# %timeit matsolver(A,B)
# # print(np.linalg.solve(A, B))
# %timeit np.linalg.solve(A, B)
# A = [[0,1,2],[2,4,-3],[3,6,-5]]
# print(A)
# print(det(A))
# print(A)
#A = np.random.rand(10,10).tolist()
#b = np.random.rand(10,1).tolist()

%timeit np.linalg.solve(np.array(A), np.array(b)).tolist()

%timeit matsolver(A,b)

* <h5>The method I have choosen to solve this problem is by using the property, if Ax=b then x=A$^{-1}$b, and hence for this method I have to first calculate A$^{-1}$</h5>

    * To calculate inverse of matrix A, I had to calculate its adjoint and the determinant of the matrix A
    

* Method to calculate determinant of matrix A
    1. To calculate the determinant of any nxn matrix I first need the co-factor for the respective elements in first row, so therefor I need determinant of minor matrix formed by deleting i row and j column and get it's determinant, this will be multiplied if elements in first row and I will get determinant of overall matrix A
    1. So if the matrix is a 1x1 return the element as that is it's determinant
    1. if determinat is 2x2 we know the mathematical formula to caculate its determinant, I have used the same
    1. then for matrix bigger then 2x2, I used reccursive relation to calculate its determinant by finding detrminant of minor matrix hence the cofactors and get the determinant by multiplying the elements of first row with it.
    

* Method to caclulate minor matrix
    1. So therfore first thing to do is make a function that can give me a matrix after deleting i'th row and j'th column, for that I take three arguments they are matrix A and element position whose minor matrix I need
    1. Next I have copied the matrix A to matrix B, I have done this because I dont want to change the matrix A, as this could later create problem in recursion or some other function I might need the A as it is.
    1. Then simply using del function to delete the i'th row is easy part as matrix in python is just a nested list
    1. Tricky part is to delete the j'th column,for that I used a for loop to iterate through each nested list and deleted the j'th element; the for loop takes a row from the matrix and then deletes the j'th element from that row and performing this action on all rows effectively deletes j'th column


* Method to calculate transpose of matrix A.
    1. To find inverse of a matrix A also need to calculate the transpose of the matrix obtained by it's co-facotors
    1. Making transpose of a matrix is mathematically means to exchange ij element to ji element.
    1. To do so I have first made a zero matrix of same size of matrix A, then changing each element for ij to ji using two for loops, in case of i=j the element remains at the same postion.